In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import os
import random

2022-12-27 14:44:15.625056: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
DIRECTORY = r"/Users/anisurrahman/Documents/ML/Deep Learning/CNN/Covid/train"
TEST = r"/Users/anisurrahman/Documents/ML/Deep Learning/CNN/Covid/test"

CATEGORIES = ["Covid","Normal","Viral Pneumonia"]

data = []
for category in CATEGORIES:
    folder = os.path.join(DIRECTORY,category)
    label = CATEGORIES.index(category)
#     print(folder)
    for img in os.listdir(folder):
        img = os.path.join(folder,img)
        img_arr = cv2.imread(img)
        img_arr = cv2.resize(img_arr,(100,100))
#         plt.imshow(img_arr)
#         print(img_arr)
        data.append([img_arr,label])
        

In [3]:
def get_label(index):
    if index == 0:
        return "Covid"
    if index == 1:
        return "Normal"
    if index == 2:
        return "Viral Pneumonia"
    
def show_pic(index): 
    plt.imshow(data[index][0])
    plt.xlabel(get_label(data[index][1]))
    plt.show()
    
# show_pic(5)
# show_pic(1)


In [4]:
random.shuffle(data)
# show_pic(2)

In [5]:
X = []
Y = []

for features, label in data:
    X.append(features)   
    Y.append(label)
    
    
X = np.asarray(X)
Y = np.asarray(Y)

X = X / 255
X.shape[1:]

(100, 100, 3)

In [6]:
IMAGE_SIZE = 224
BATCH_SIZE = 64

#pre=processing
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.1
    )

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
     rescale=1./255,
     validation_split=0.1
)

train_datagen = train_datagen.flow_from_directory(
    DIRECTORY,
    target_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    subset = 'training'
)

test_datagen = test_datagen.flow_from_directory(
    TEST,
    target_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    subset = 'validation'
)

Found 226 images belonging to 3 classes.
Found 6 images belonging to 3 classes.


In [7]:
train_datagen.class_indices


{'Covid': 0, 'Normal': 1, 'Viral Pneumonia': 2}

In [8]:

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense

model = Sequential()
model.add(Conv2D(filters=32,padding='same',strides=2,kernel_size=3,activation='relu',input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=2,strides=2))

model.add(tf.keras.layers.Conv2D(filters=32,padding='same',strides=2,kernel_size=3,activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

# cnn.add(tf.keras.layers.Conv2D(filters=32,padding='same',strides=2,kernel_size=3,activation='relu'))
# cnn.add(tf.keras.layers.MaxPool2D(pool_size=2))

model.add(Flatten())
model.add(Dense(3,activation='softmax'))

2022-12-27 14:44:21.931743: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

history = model.fit(train_datagen,epochs=10,validation_data=test_datagen)

Epoch 1/10
4/4 [==============================] - 7s 2s/step - loss: 1.0695 - accuracy: 0.4646 - val_loss: 0.9839 - val_accuracy: 0.6667
Epoch 2/10
4/4 [==============================] - 6s 2s/step - loss: 0.9471 - accuracy: 0.6195 - val_loss: 0.8775 - val_accuracy: 0.6667
Epoch 3/10
4/4 [==============================] - 6s 2s/step - loss: 0.8371 - accuracy: 0.6416 - val_loss: 0.7898 - val_accuracy: 0.6667
Epoch 4/10
4/4 [==============================] - 6s 1s/step - loss: 0.6858 - accuracy: 0.7522 - val_loss: 0.5524 - val_accuracy: 1.0000
Epoch 5/10
4/4 [==============================] - 6s 1s/step - loss: 0.5900 - accuracy: 0.7788 - val_loss: 0.5418 - val_accuracy: 0.6667
Epoch 6/10
4/4 [==============================] - 6s 2s/step - loss: 0.4710 - accuracy: 0.8142 - val_loss: 0.3348 - val_accuracy: 1.0000
Epoch 7/10
4/4 [==============================] - 6s 1s/step - loss: 0.4713 - accuracy: 0.8009 - val_loss: 0.5202 - val_accuracy: 0.6667
Epoch 8/10
4/4 [=========================

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.models import Model
a = np.array(history.history['accuracy']).mean()
# history.history
model.evaluate_generator(generator=test_datagen)

In [ ]:
from keras.preprocessing import image
path = "/Users/anisurrahman/Documents/ML/Deep Learning/CNN/Covid/net"
# path = "/home/user/ML/Deep Learning/CNN/Covid/test/Normal"
img = path + "/covid.png"
try:
    
    img_pred = tf.keras.utils.load_img(img,target_size=(IMAGE_SIZE,IMAGE_SIZE))
    img_pred = tf.keras.utils.img_to_array(img_pred)
    img_pred = img_pred / 255
    img_pred = np.expand_dims(img_pred, axis=0)
except:
    print("file not found!")

result = model.predict(img_pred)
# plt.imshow(img_pred)
print(result)
# print(get_label(np.argmax(result)))
print(CATEGORIES[np.argmax(result)])

In [ ]:
# model.evaluate(X,Y)